In [2]:
from astropy.nddata import NDDataArray as nda, StdDevUncertainty as sdu
from astropy.table import Table, column
import astropy.units as u
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
# better-looking plots
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 5
mpl.ticker.AutoLocator.default_params['prune'] = 'both'

mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

import sys
sys.path.append('../astro_codes/')

#### First task is sample selection:

In [3]:
gz2_data = Table.read('../fits/full_sample_debiased_w_low_z_mod.fits')
axial_data = Table.read('../fits/Axial_ratios/dr7_isoAB_matched.fits')

In [4]:
morph_data = Table()
morph_data['dr7objid'] = gz2_data['dr7objid']

p_spiral = (gz2_data['t01_smooth_or_features_a02_features_or_disk_debiased_rh']
          * gz2_data['t02_edgeon_a05_no_debiased_rh']
          * gz2_data['t04_spiral_a08_spiral_debiased_rh'])

N_spiral = (gz2_data['t04_spiral_a08_spiral_count'] - 
            gz2_data['t11_arms_number_a37_cant_tell_count'])

p_bar = gz2_data['t03_bar_a06_bar_debiased_rh']

m_columns = ('t11_arms_number_a31_1_debiased_rh',
             't11_arms_number_a32_2_debiased_rh',
             't11_arms_number_a33_3_debiased_rh',
             't11_arms_number_a34_4_debiased_rh',
             't11_arms_number_a36_more_than_4_debiased_rh')

m_array = np.array([gz2_data[c] for c in m_columns]).T
m_absolute = np.argmax(m_array,axis=1) + 1
m_absolute_max = np.max(m_array,axis=1)
m_avg = np.zeros(len(m_array))
for c in range(5):
    m_avg += (c+1) * m_array[:,c]
m_avg /= np.sum(m_array,axis=1)
m_avg[np.isfinite(m_avg) == False] = 0

morph_data['redshift'] = gz2_data['REDSHIFT_1']
morph_data['axial_ratio'] = axial_data['isoB_g'] / axial_data['isoA_g']
morph_data['p_spiral'] = p_spiral
morph_data['N_spiral'] = N_spiral
morph_data['p_bar'] = p_bar
morph_data['m_avg'] = m_avg
morph_data['m'] = m_absolute
morph_data['p_m'] = m_absolute_max
morph_data.write('fits/morph_data.fits',overwrite=True)

/home/ppxrh/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:25: RuntimeWarning: invalid value encountered in true_divide


#### Galaxy structure data (mass + size)

In [4]:
from combining_catalogues import match_ids, make_matched_catalogue

simard_data = Table.read('../fits/Simard+Mendel/simard.fits')
mendel_data = Table.read('../fits/Simard+Mendel/mendel_dusty.fits')

In [5]:
ids_ = match_ids(gz2_data,simard_data,'dr7objid','objID')
simard_matched = make_matched_catalogue(gz2_data,simard_data,ids_)
simard_matched['dr7objid'] = gz2_data['dr7objid']
simard_matched.write('../fits/Simard+Mendel/simard_matched.fits',
                     overwrite=True)

ids_ = match_ids(gz2_data,mendel_data,'dr7objid','objID')
mendel_matched = make_matched_catalogue(gz2_data,mendel_data,ids_)
mendel_matched['dr7objid'] = gz2_data['dr7objid']
mendel_matched.write('../fits/Simard+Mendel/mendel_matched.fits',
                     overwrite=True)

248883 galaxies in the reference catalogue
1123718 galaxies in the match catalogue
---> 245259 matches in total
248883 galaxies in the reference catalogue
657996 galaxies in the match catalogue
---> 237996 matches in total


In [8]:
structure_data = Table()
structure_data['dr7objid'] = gz2_data['dr7objid']


fit_ok = np.all([mendel_matched['Type'] != 4,mendel_matched['Type'] != -999,
                 simard_matched['Re'] > 0, simard_matched['Rd'] > 0],axis=0)

def mass_and_error(data,columns):
    mass = 10**data[columns[0]]
    mass_low = 10**data[columns[1]]
    mass_high = 10**data[columns[2]]
    mass_error = np.max([mass-mass_low,mass_high-mass],axis=0)
    #mass_error = (mass_high - mass_low)/2
    mass[(mass <= 0) | (np.isfinite(mass) == False)] = 1
    mass_error[(mass_error <= 0) | (np.isfinite(mass_error) == False)] = 1 
    return mass, mass_error

m_b, m_b_error = mass_and_error(mendel_matched,['logMb','b_logMb','b_logmb_lc'])
m_d,m_d_error = mass_and_error(mendel_matched,['logMd','b_logMd','b_logmd_lc'])
m_total = nda(m_d,sdu(m_d_error)).add(nda(m_b,sdu(m_b_error)))
m_t, m_t_error = m_total.data, m_total.uncertainty.array

R_b, R_b_error = (simard_matched['Re'], simard_matched['e_Re'])
R_d, R_d_error = (simard_matched['Rd'], simard_matched['e_Rd'])

structure_data['m_b'] = m_b * u.Msun
structure_data['delta_m_b'] = m_b_error * u.Msun
structure_data['m_d'] = m_d * u.Msun
structure_data['delta_m_d'] = m_d_error * u.Msun
structure_data['m_t'] = m_t * u.Msun
structure_data['delta_m_t'] = m_t_error * u.Msun#
structure_data['R_b_r'] = R_b
structure_data['delta_R_b_r'] = R_b_error
structure_data['R_d_r'] = R_d
structure_data['delta_R_d_r'] = R_d_error
structure_data['Type'] = mendel_matched['Type']
structure_data['ok'] = fit_ok
structure_data['PpS_mendel'] = mendel_matched['PpS']

structure_data.write('fits/structure_data.fits',overwrite=True)

/home/ppxrh/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in greater
/home/ppxrh/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:14: RuntimeWarning: invalid value encountered in less_equal
/home/ppxrh/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in less_equal


#### Final data: HI data

In [9]:
a70_data = Table.read('../fits/SF_fits/a70.fits')
teimoorinia_data = Table.read('../fits/ALFALFA/teimoorinia_hi.fits')

In [10]:
# match up the Teimoorinia data to the GZ2 data
from combining_catalogues import match_ids, make_matched_catalogue

matched_ids = match_ids(gz2_data,teimoorinia_data,'dr7objid','ID_SDSS_')
teimoorinia_matched = make_matched_catalogue(gz2_data,teimoorinia_data,matched_ids)
teimoorinia_matched['dr7objid'] = gz2_data['dr7objid']

teimoorinia_matched.write('fits/teimoorinia_matched.fits',overwrite=True)

248883 galaxies in the reference catalogue
561585 galaxies in the match catalogue
---> 225382 matches in total


In [11]:
hi_data = Table()
hi_data['dr7objid'] = gz2_data['dr7objid']

m_hi_t17 = (teimoorinia_matched['_logM_HI/M_*_'] 
          + teimoorinia_matched['_logM_*_'])

hi_data['m_hi_t17'] = 10**m_hi_t17
hi_data['cfgas'] = teimoorinia_matched['_C_fgas_']
#hi_data['delta_m_hi_t17'] = 0.22 * hi_data['m_hi_t17']
hi_data['delta_m_hi_t17'] = 2.303 * hi_data['m_hi_t17'] * 0.22

hi_data['detcode'] = a70_data['detcode']

hi_data['m_hi_a70'] = 10**a70_data['loghimass']
hi_data['delta_m_hi_a70'] = 10**a70_data['loghimass'] / a70_data['snratio']

a70_ok = a70_data['loghimass'] > 0

hi_data['hi_mass'] = hi_data['m_hi_t17']
hi_data['hi_mass_error'] = hi_data['delta_m_hi_t17']

hi_data['hi_mass'][a70_ok] = hi_data['m_hi_a70'][a70_ok]
hi_data['hi_mass_error'][a70_ok] = hi_data['delta_m_hi_a70'][a70_ok]

hi_data.write('fits/hi_data.fits',overwrite=True)